In [1]:
from utils import CIFAR10Data, plot_costs_accuracies
import numpy as np
import scipy
import pickle
import matplotlib.pyplot as plt

## K-Layer Classifier

In [ ]:
class Layer:
    def __init(self, input_dim, output_dim, activation='relu'):
        self.W = np.random.normal(loc=0, scale=1/np.sqrt(input_dim), size=(output_dim, input_dim)) 
        self.b = np.zeros((output_dim,1))
        self.activation = activation # should be softmax for last layer in definition
#         self.eta = eta
        pass
    
    def activation(self, S):
        if self.activation == 'relu':
            return self.relu(S) 
        elif self.activation == 'softmax':
            return self.softmax()
        else:
            raise Exception('activation function invalid') 
    
    def relu(self, X):
        return np.where(X >= 0, X, 0)
    
    
    def softmax(self, X):
        return np.exp(X) / np.sum(np.exp(X), axis=0)
    
    
    def forward_pass(self, X, last_layer=False):
        S = np.dot(self.W, X) + self.b
        return self.activation(S)
        
    def backward_pass(self):
        pass
    
    def update_gradients(self, grad_W, grad_b, eta):
        grad_W -= self.eta * 
    

In [18]:
class KLayerClassifer:
    def __init__(self, input_dim, batch_size=100, eta=0.001, n_epochs=20, lamda=0, cyclical_lr=False, n_s=None, eta_min=None, eta_max=None, lr_range_test=False, dropout_thresh=None):
        n_classes = 10
        self.batch_size = batch_size # number of images in a batch
        self.n_epochs = n_epochs
        self.lamda = lamda
        
        self.cyclical_lr = cyclical_lr
        if cyclical_lr:
            self.batch_size = 100
            self.l = 0
            self.n_s = n_s # step size
            self.eta_min = eta_min
            self.eta_max = eta_max
            self.plot_every_n_steps = int(2*n_s/10)
        else:
            self.eta = eta
        self.lr_range_test = lr_range_test # whether to perform learning rate range test to see what eta_min and eta_max should be
        self.plot_every_n_steps = batch_size 
        self.n_hidden = n_hidden
        self.dropout_thresh = dropout_thresh # proportion of nodes to switch off ( =(1-p) from slides)
        
        # initialise layers
        self.layers = None
    
    def add_layer(self, input_dim, output_dim, activation='relu'):
        # add a new layer
        if self.layers is None:
            self.layers = []
        self.layers.append(Layer(input_dim, output_dim, activation))
        

    def normalise(self, train_X, val_X, test_X):
        ''' X has shape (d,n) where d = dimensionality of each image, n is number of images '''
        mean = np.mean(train_X, axis=1)
        std = np.std(train_X, axis=1)
        original_shape = train_X.shape
        # apply same transformation to all of the datasets using params from train set
        def _normalise_helper(a, m, s):
            return ((a.T - m.T) / s.T).T
        
        train_X = _normalise_helper(train_X, mean, std)
        val_X = _normalise_helper(val_X, mean, std)
        test_X = _normalise_helper(test_X, mean, std)
        return train_X, val_X, test_X

    
    def forward_pass(self, X):
        H = np.copy(X)
        Xs = []
        for layer in self.layers[:-1]: # loop layers 1...k-1
            H = layer.forward_pass(H)
            Xs.append(H)
        # apply softmax to last layer instead of relu
        P = layer.forward_pass(H, last_layer='True')
        assert len(Xs) = len(self.layers) - 1 # k -1
        return Xs, P


    def compute_accuracy(self, X, Y):
        ''' X is data (dim, N), y is gt (C, N), W is weight matrix, b is bias, Y is 1hot encoded labels'''
        _, P = self.forward_pass(X)
        pred = np.argmax(P, axis=0)
        lbls = np.argmax(Y, axis=0)
        accuracy = np.mean(pred == lbls)
        return pred, accuracy
    

    def compute_cost(self, X, Y, W1, W2, b1, b2): # TODO
        ''' 
            X: dxn (dimensionality by # images)
            Y: Kxn (no. classes one-hot encoded by # images)
            J: scalar corresponding to sum of loss of ntwks predictions of X relative to gt labels 
        '''
        _, P = self.forward_pass(X)
        N = X.shape[1]
        loss = -np.sum(Y*np.log(P)) / N
        cost = loss + self.lamda * (np.sum(W1**2) + np.sum(W2**2))
    
        return loss, cost
        
    
    
    def compute_gradients(self, X, Y):
        ''' computes gradients of the cost function wrt W and b for batch X '''
        N = X.shape[1]

        # forward pass, apply dropout if its set
        Xs, P = self.forward_pass(X)
        
        # backward pass
        G = -(Y - P)
        k = len(self.layers)
        
        grad_Ws = []
        grad_bs = []
        
        for l in range(k-1, 1, -1): # propagate gradient backwarsd to first layer
            layer = self.layers[l]
            H = Xs[l-1]
            grad_W = np.dot(G, H.T) / N + 2 * layer.lamda * layer.W 
            grad_b = np.sum(G, axis=1) / N
            grad_Ws.append(grad_W)
            grad_bs.append(grad_b)
            G = np.dot(layer.W.T, G)
            Ind = H > 0
            G = np.multiply(G, Ind)
            
        # For first layer
        layer = self.layers[0]
        grad_W = np.dot(G, X.T) / N + 2 * layer.lamda * layer.W 
        grad_b = np.sum(G, axis=1) / N
        grad_Ws.append(grad_W)
        grad_bs.append(grad_b)
        
        return grad_Ws, grad_bs
    
   
    def compute_cyclical_lr(self, t):
        l = self.l
        eta_min = self.eta_min
        eta_max = self.eta_max
        n_s = self.n_s
        if t >= 2*l*n_s and t <= (2*l+1)*n_s:
            eta = eta_min + (t-2*l*n_s)*(eta_max-eta_min)/n_s
        elif t >= (2*l+1)*n_s and t <= 2*(l+1)*n_s:
            eta = eta_max - (t - (2*l+1)*n_s)*(eta_max-eta_min)/n_s
            
        if t % (2*n_s) == 0 and t > 0: # update l every 2*n_s steps
                self.l += 1
        return eta
    
        
        
    def train(self, X, Y, random_shuffle=False, val_X=None, val_Y=None, get_accuracies_costs=False, max_lr=0.1):
        n = X.shape[1]
        
        number_of_batches = int(n / self.batch_size)
        assert number_of_batches > 0
        indices = np.arange(X.shape[1])
        if random_shuffle:
            print('Randomly shuffling')
        
        if self.cyclical_lr:
            print('Cyclical learning rate')
       
        accuracies = {'train': [], 'val': []}
        costs = {'train': [], 'val': []}
        losses = {'train': [], 'val': []}
        update_steps = []
        
        eta = self.eta if not self.cyclical_lr else None
        t = 0 # each update step % 2*n_s
        if self.lr_range_test:
            eta = 0
            
            self.plot_every_n_step = int(self.n_epochs * number_of_batches / 20)
            eta_incr = max_lr * self.plot_every_n_step / (self.n_epochs * number_of_batches)
            
        
        for epoch in range(self.n_epochs):     
            if random_shuffle:
                np.random.shuffle(indices)
                X = np.take(X, indices, axis=1)
                Y = np.take(Y, indices, axis=1)
                
            for j in range(number_of_batches):
                if self.cyclical_lr:
                    eta = self.compute_cyclical_lr(t)
                
                
                j_start = j * self.batch_size
                j_end = (j+1) * self.batch_size
                Xbatch = X[:, j_start:j_end]
                Ybatch = Y[:, j_start:j_end]
                
                # Perform MiniBatch Gradient Descent
                grad_Ws, grad_bs = self.compute_gradients(Xbatch, Ybatch)
                for i in range(len(self.layers)): 
                    self.layers[i].update_gradients(grad_Ws[i], grad_bs[i], self.eta) 

                if get_accuracies_costs and (t % self.plot_every_n_steps == 0):
                    _, train_accuracy = self.compute_accuracy(X, Y)
                    _, val_accuracy = self.compute_accuracy(val_X, val_Y)
                    accuracies['train'].append(train_accuracy)
                    accuracies['val'].append(val_accuracy)
                    train_loss, train_cost = self.compute_cost(X, Y)
                    val_loss, val_cost = self.compute_cost(val_X, val_Y)
                    costs['train'].append(train_cost)
                    costs['val'].append(val_cost)
                    losses['train'].append(train_loss)
                    losses['val'].append(val_loss)
                    update_steps.append(t)
            
                
                if self.lr_range_test and (t % self.plot_every_n_step == 0):
                    update_steps.append(eta)
                    _, train_accuracy = self.compute_accuracy(X, Y)
                    _, val_accuracy = self.compute_accuracy(val_X, val_Y)
                    accuracies['train'].append(train_accuracy)
                    accuracies['val'].append(val_accuracy)
                    eta += eta_incr
                    
                t += 1
   

        return accuracies, losses, costs, update_steps
 

In [6]:
# load in data and normalise
CIFARDATA = CIFAR10Data(dataset_dir='../datasets/cifar-10-batches-py/')
train_X, train_Y = CIFARDATA.load_batch('data_batch_1')
val_X, val_Y = CIFARDATA.load_batch('data_batch_2')
test_X, test_Y = CIFARDATA.load_batch('test_batch')
datasets = [train_X, train_Y, val_X, val_Y, test_X , test_Y]

Let's check gradients are correct for 3 layer classifier

## Batch normalisation

## Optimising the network

Do a more exhaustive random search to find good values for the amount of regularization.

Apply dropout to your training if you have a high number of hidden nodes and you feel you need more regularization.

It has been empirically reported in several works that you get better perfor- mance by the final network if you apply batch normalization to the scores after the non-linear activation function has been applied. You could inves- tigate whether this is the case. You will have to update your forward and backward pass of the back-prop algorithm accordingly.

Do a more thorough search to find a good network architecture. Does making the network deeper improve performance?